In [ ]:
from pathlib import Path
import ast
import pandas as pd
import numpy as np
from tqdm import tqdm
import subprocess
import warnings
from joblib import Parallel, delayed
import os
import multiprocessing
import time
import random

In [44]:
from truckfactor.compute import main
import os
import warnings
from pathlib import Path
import pandas as pd
import numpy as np
from pandarallel import pandarallel
import random
import glob
import subprocess
from pygit2 import Object, Repository, GIT_SORT_TIME, enums, init_repository, Patch

In [22]:
warnings.filterwarnings("ignore")
pandarallel.initialize(progress_bar = True)

pr_indir = Path('drive/output/scrape/push_pr_commit_data/pull_request_data')
commits_outdir = Path('drive/output/scrape/collect_commits')

df_pull_request = importPullRequestData(pr_indir)

github_repos = df_pull_request['repo_name'].unique().tolist()
github_repos = [library for library in github_repos if "/" in library]
random.shuffle(github_repos)

for library in github_repos:
    print(library)

print("Done!")

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


nikipore/stompest
4dn-dcic/Submit4DN
internetarchive/surt
ktosiek/pytest-freezegun
perrygeo/python-rasterstats
martyzz1/heroku3.py
zostera/django-bootstrap3
fedora-infra/fedmsg_meta_fedora_infrastructure
saffsd/langid.py
MichaelYusko/GitterPy
onnx/onnx
pavoni/pyroon
wadda/gps3
jedie/bootstrap_env
spyder-ide/pywinpty
imjoey/pyhaproxy
Infinidat/infi.wioctl
bmcfee/resampy
abraia/abraia-python
greyli/flask-dropzone
gmr/pamqp
guykisel/robotframework-faker
ronaldoussoren/modulegraph
taxjar/taxjar-python
gregoil/rotest
fcurella/cookiejar
google/python_portpicker
upsight/doctor
cherrypy/cheroot
atdt/monotonic
singularityhub/singularity-cli
jsocol/django-ratelimit
data61/clkhash
beardypig/pymp4
seebass/drf-enum-field
brutasse/django-password-reset
dsoprea/PyInotify
Kami/python-yubico-client
underyx/structlog-pretty
alekstorm/backports.ssl
zopefoundation/zope.i18nmessageid
note35/sinon
rtlee9/serveit
zopefoundation/zope.app.dependable
ashleysommer/sanic-cors
netbox-community/pynetbox
adafruit/Ad

In [40]:
print(library)
lib_name = library.split("/")[1]
lib_renamed = library.replace("/","_")
subprocess.Popen(["git", "clone", f"git@github.com:{library}.git", f"{lib_renamed}"], cwd = '../github_repos').communicate()

global repo
repo = Repository(f'../github_repos/{lib_renamed}')
latest_commit = repo[repo.head.target]

data_commits = [[commit.commit_time, commit.id] for commit in repo.walk(latest_commit.id, enums.SortMode.TIME)]
df_commits = pd.DataFrame(data_commits, columns = ['time','commit_sha'])
df_commits['date'] = pd.to_datetime(df_commits['time'], unit = 's')
df_commits['week'] = df_commits['date'].apply(lambda x: x.week)
df_commits['year'] = df_commits['date'].apply(lambda x: x.year)
df_commits = df_commits.sort_values('date', ascending = True).drop_duplicates(['week','year'])
df_commits = df_commits.sort_values('date', ascending = False)
df_commits.reset_index(drop = True, inplace = True)

for commit_num in df_commits.index:
    subprocess.Popen(["git", "reset", "--hard", str(df_commits.loc[commit_num, 'commit_sha'])], cwd = f'../github_repos/{lib_renamed}').communicate()
    truckfactor, _, authors = main(f'../github_repos/{lib_renamed}')
    df_commits.loc[commit_num, ['truckfactor','authors']] = [truckfactor, authors]

jambonrose/markdown_superscript_extension


Cloning into 'jambonrose_markdown_superscript_extension'...
Updating files: 100% (38/38), done.


(None, None)

In [93]:
for commit_num in df_commits.index:
    subprocess.Popen(["git", "reset", "--hard", str(df_commits.loc[commit_num, 'commit_sha'])], cwd = f'../github_repos/{lib_renamed}').communicate()
    truckfactor, _, authors = main(f'../github_repos/{lib_renamed}')
    df_commits.loc[commit_num, ['truckfactor']] = [truckfactor]
    df_commits.loc[commit_num, ['authors']] = [truckfactor]

HEAD is now at 82e5001 ❗️  Scheduled monthly dependency update for December (#48)
The truck factor of ../github_repos/jambonrose_markdown_superscript_extension (82e500182036fd754cd12cb1a3a7f71e2eeb05b1) is: 1
with author(s): Andrew Pinkham
HEAD is now at 36382ad 🔨 Fix typos & misconfigs (2018-10-09) (#43)
The truck factor of ../github_repos/jambonrose_markdown_superscript_extension (36382ad4f04f3318b9016aadbcc068151db3c57b) is: 1
with author(s): Andrew Pinkham
HEAD is now at 07ec818 ❗️ Scheduled monthly dependency update for October (#40)
The truck factor of ../github_repos/jambonrose_markdown_superscript_extension (07ec8182e41376730835faea56ce0929bc5bc29c) is: 1
with author(s): Andrew Pinkham
HEAD is now at 4ccd197 ❗️ Update tox from 3.0.0 to 3.1.2 (#38)
The truck factor of ../github_repos/jambonrose_markdown_superscript_extension (4ccd197e3d1b2b7f776165ebb4126d15baf02c97) is: 1
with author(s): Andrew Pinkham
HEAD is now at 7ef5dd0 Update wheel from 0.31.0 to 0.31.1 (#37)
The truck fa

In [89]:
subprocess.Popen(["git", "reset", "--hard", str(df_commits.loc[2, 'commit_sha'])], cwd = f'../github_repos/{lib_renamed}').communicate()
truckfactor, commit_sha, authors = main(f'../github_repos/{lib_renamed}')
print(truckfactor, commit_sha, authors)

HEAD is now at 07ec818 ❗️ Scheduled monthly dependency update for October (#40)
The truck factor of ../github_repos/jambonrose_markdown_superscript_extension (07ec8182e41376730835faea56ce0929bc5bc29c) is: 1
with author(s): Andrew Pinkham
1 None ['Andrew Pinkham']


In [ ]:
get weekly commits
for each commit in each week
reset to that commit (go in order from most to least recent)
calculate and save truck factor

In [ ]:
def cleanCommitData(library, df_library, lib_renamed):
    subprocess.Popen(["git", "clone", f"git@github.com:{library}.git", f"{lib_renamed}"], cwd = commits_outdir / 'github_repos').communicate()

    global repo
    df_library.reset_index(drop = True, inplace = True)
    df_library_index = list(df_library.index)
    df_library_chunks = [df_library_index[x:x+3000] for x in range(0, len(df_library_index), 3000)]

    print(len(df_library_index))
    commit_data = pd.Series()
    i = 0
    df_commit_groups = pd.DataFrame(columns = ['repo_name', 'pr_number', 'pr_commits_url', 'parent_commit', 'commit_groups'])
    for chunk in df_library_chunks:
        start = time.time()
        cloned_repo_location = Path(commits_outdir / 'github_repos' / lib_renamed)
        repo = Repository(cloned_repo_location)
        df_library_chunk = df_library.loc[chunk]
        df_library_chunk['parent_commit'] = df_library_chunk.parallel_apply(lambda x: getHead(x['commit_list'], x['pr_number'], cloned_repo_location) if not pd.isnull(x['pr_number']) else np.nan, axis = 1)
        print(f"finished getting parent commits for {library}, chunk {i+1}")
        df_library_chunk['commit_groups'] = df_library_chunk.parallel_apply(lambda x: createCommitGroup(x['commit_list'], x['parent_commit']) if not pd.isnull(x['parent_commit']) else np.nan, axis = 1)
        df_commit_groups_chunk = df_library_chunk[['repo_name', 'pr_number', 'pr_commits_url', 'parent_commit', 'commit_groups']].explode('commit_groups')
        df_commit_groups = pd.concat([df_commit_groups, df_commit_groups_chunk])

        commit_data_chunk = df_commit_groups_chunk['commit_groups'].parallel_apply(lambda x: returnCommitStats(x, repo) if type(x) != float else x)
        commit_data = pd.concat([commit_data, commit_data_chunk])
        print(f"NOW REMOVING {lib_renamed}")
        while lib_renamed in os.listdir(commits_outdir / 'github_repos'):
            subprocess.Popen(["rm", "-rf", f"{lib_renamed}"], cwd = commits_outdir / 'github_repos').communicate()
        if i != len(df_library_chunks)-1:
            print(f"COPYING {lib_renamed}_temp to {lib_renamed}")
            subprocess.Popen(["cp", "-r", f"{lib_renamed}_temp", f"{lib_renamed}"], cwd = commits_outdir / 'github_repos').communicate()    
        i+=1
        end = time.time()
        print(end - start)

    print(f"NOW REMOVING {lib_renamed}_temp")
    while f"{lib_renamed}_temp" in os.listdir(commits_outdir / 'github_repos'):
        subprocess.Popen(["rm", "-rf", f"{lib_renamed}_temp"], cwd = commits_outdir / 'github_repos').communicate()

    commit_data_colnames = ['commit sha', 'commit author name', 'commit author email', 'committer name',
                                       'commmitter email', 'commit message', 'commit additions', 'commit deletions',
                                       'commit changes total', 'commit files changed count', 'commit file changes', 
                                       'commit time']
    df_commit = pd.DataFrame(commit_data.apply(lambda x: [np.nan]* len(commit_data_colnames) if type(x) == float else x).tolist(),
                            columns = commit_data_colnames)

    # In[ ]:
    df_commit_final = pd.concat([df_commit_groups.reset_index(drop = True), df_commit], axis = 1)
    df_commit_final['pr_number'] = pd.to_numeric(df_commit_final['pr_number'])
    df_commit_final = df_commit_final.dropna()

    return df_commit_final


def GetTruckFactor(library, truckfactor_outdir, df_library):
    print(library)
    lib_name = library.split("/")[1]
    lib_renamed = library.replace("/","_")
    if f'truckfactor_{lib_renamed}.csv' not in os.listdir(truckfactor_outdir):
        try:
            print(f"Starting {library}")
            start = time.time()
            if lib_renamed not in os.listdir(commits_outdir / 'github_repos'):
                df_commit_final = cleanCommitData(library, df_library, lib_renamed)
                df_commit_final.to_csv(commits_outdir / f'commits_pr_{lib_renamed}.csv')
                end = time.time()
                print(f"{library} completed in {start - end}")
            else:
                print(f"skipping {lib_renamed} because it's a concurrent process")
            return "success"
        except Exception as e:
            print(e)
            return f"failure, {str(e)}"
    return 'success'

